# 환경설정

In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

In [2]:
with open('df_scaled_UTD.pickle','rb') as f:
    df_scaled_UTD = pickle.load(f)
with open('sector.pickle','rb') as f:
    sector = pickle.load(f)

## 파라미터 지정

In [3]:
#윈도우 사이즈
WINDOW_SIZE = 10
# X에 대한 윈도우 함수 설정값
X_size = WINDOW_SIZE
X_shift = 10
X_stride = 1
# y에 대한 윈도우 함수 설정값
y_size = 1
y_shift = WINDOW_SIZE
y_stride = 1

batch_size = 1

# 2. 모델 불러오기

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
model_dic ={}
for sector_name in sector.keys():
    model_dic[sector_name] = Sequential([
        # 1차원 feature map 생성
        Conv1D(filters=32, kernel_size=5,
               padding="causal",
               activation="relu",
               #feature의 개수 3개
               input_shape=[WINDOW_SIZE, 4]),
        # LSTM
        LSTM(16, activation='tanh'),
        Dense(16, activation="relu"),
        Dense(1),
    ])

In [6]:
with open('filename_dic.pickle', 'rb') as f:
    filename_dic = pickle.load(f)

In [7]:
filename_dic

{'Food': 'Models\\Foodckeckpointer.ckpt',
 'Clothing': 'Models\\Clothingckeckpointer.ckpt',
 'Chemical': 'Models\\Chemicalckeckpointer.ckpt',
 'Medicine': 'Models\\Medicineckeckpointer.ckpt',
 'Non_Metal': 'Models\\Non_Metalckeckpointer.ckpt',
 'Metal': 'Models\\Metalckeckpointer.ckpt',
 'Machine': 'Models\\Machineckeckpointer.ckpt',
 'Electronic': 'Models\\Electronicckeckpointer.ckpt',
 'Construction': 'Models\\Constructionckeckpointer.ckpt',
 'Transport': 'Models\\Transportckeckpointer.ckpt',
 'Distribution': 'Models\\Distributionckeckpointer.ckpt',
 'Power': 'Models\\Powerckeckpointer.ckpt',
 'Tele': 'Models\\Teleckeckpointer.ckpt',
 'Finance': 'Models\\Financeckeckpointer.ckpt',
 'Brokerage': 'Models\\Brokerageckeckpointer.ckpt',
 'Insurer': 'Models\\Insurerckeckpointer.ckpt',
 'Service': 'Models\\Serviceckeckpointer.ckpt',
 'Manufacturer': 'Models\\Manufacturerckeckpointer.ckpt'}

In [8]:
for_linux = {}
for sector_name in filename_dic:
    for_linux[sector_name] = filename_dic[sector_name][7:]

In [9]:
for_linux

{'Food': 'Foodckeckpointer.ckpt',
 'Clothing': 'Clothingckeckpointer.ckpt',
 'Chemical': 'Chemicalckeckpointer.ckpt',
 'Medicine': 'Medicineckeckpointer.ckpt',
 'Non_Metal': 'Non_Metalckeckpointer.ckpt',
 'Metal': 'Metalckeckpointer.ckpt',
 'Machine': 'Machineckeckpointer.ckpt',
 'Electronic': 'Electronicckeckpointer.ckpt',
 'Construction': 'Constructionckeckpointer.ckpt',
 'Transport': 'Transportckeckpointer.ckpt',
 'Distribution': 'Distributionckeckpointer.ckpt',
 'Power': 'Powerckeckpointer.ckpt',
 'Tele': 'Teleckeckpointer.ckpt',
 'Finance': 'Financeckeckpointer.ckpt',
 'Brokerage': 'Brokerageckeckpointer.ckpt',
 'Insurer': 'Insurerckeckpointer.ckpt',
 'Service': 'Serviceckeckpointer.ckpt',
 'Manufacturer': 'Manufacturerckeckpointer.ckpt'}

In [10]:
for sector_name in filename_dic:
    model_dic[sector_name].load_weights('Models/'+for_linux[sector_name])

In [11]:
for sector_name, df in df_scaled_UTD.items():
    df_scaled_UTD[sector_name] = df.drop('종가',1)


# tomorrow 업종 상향 / 하향 예측하기
## 내일 종가 예측값  - 어제부터 10일 전 예측 종가 평균 

In [12]:

from datetime import date,datetime, timedelta
today = date.today()
tomorrow = today+timedelta(days=1, hours = 9)
tomorrow = tomorrow.isoformat()
tomorrow = tomorrow.replace('-','')[2:]
set_d = tomorrow
set_d



'220812'

In [13]:
#kr_name_list
sector_name_list = ['음식료품', '섬유의복', '화학','의약품','비금속광물','철강금속','기계','전기전자','건설업','운수창고','유통업','전기가스업','통신업', '금융업','증권','보험', '서비스업','제조업']
sector_name_mapping = {}
count = 0
for i, sector_name in enumerate(sector):
    sector_name_mapping[sector_name] = sector_name_list[i]
    count+=1


In [14]:
pred_dic = {}
for sector_name, model in model_dic.items():
    pred_l = []
    for i in range(11):
        data = tf.constant(np.expand_dims(np.array(df_scaled_UTD[sector_name].tail(10+i)[:10]),axis=0))
        pred_l.append(float(model.predict(data)))
    pre = sum(pred_l[1:])/5
    if pred_l[0]>pre:
        s = sector_name_mapping[sector_name] + '은 상향을 예상합니다.'
        print(s)
        pred_dic[sector_name] = s
    elif pred_l[0]<pre:
        s = sector_name_mapping[sector_name] + '은 하향을 예상합니다.'
        print(s)
        pred_dic[sector_name] = s
    elif pred_l[0] == pre:
        s= sector_name_mapping[sector_name]+'은 횡보를 예상합니다.'
        print(s)
        pred_dic[sector_name] = s


음식료품은 하향을 예상합니다.
섬유의복은 하향을 예상합니다.
화학은 하향을 예상합니다.
의약품은 하향을 예상합니다.
비금속광물은 하향을 예상합니다.
철강금속은 하향을 예상합니다.
기계은 하향을 예상합니다.
전기전자은 하향을 예상합니다.
건설업은 하향을 예상합니다.
운수창고은 하향을 예상합니다.
유통업은 하향을 예상합니다.
전기가스업은 하향을 예상합니다.
통신업은 하향을 예상합니다.
금융업은 하향을 예상합니다.
증권은 하향을 예상합니다.
보험은 하향을 예상합니다.
서비스업은 하향을 예상합니다.
제조업은 하향을 예상합니다.


In [15]:
with open(f'{set_d}pred_dic.pickle','wb') as f:
    pickle.dump(pred_dic,f)